In [1]:
import requests
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
import re
import nltk

In [2]:
!pip install text-generation


In [3]:
from transformers import pipeline

# Initialize the text generation pipeline with the desired model
generator = pipeline('text-generation', model='gpt2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
# Generate text based on your prompt
prompt = "machine learning"
generated_text1 = generator(prompt, max_length=50)
generated_text2 = generator(prompt, max_length=50)
generated_text3 = generator(prompt, max_length=50)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [5]:
print(generated_text1[0]['generated_text'])
print(generated_text2[0]['generated_text'])
print(generated_text3[0]['generated_text'])
generated_text1

machine learning

Using the same system that has been used ever since with Microsoft Excel, Microsoft has demonstrated a powerful way to use and create a variety of Excel documents using the Microsoft OneDrive and Microsoft OneMover. The company has started supporting the
machine learning for social networks.

That being said, I'd also like to mention how interesting this AI system is so far into its existence. It was discovered that some social networks are already interacting with each other, even with less processing time.
machine learning and cognitive-behavioral interventions for children with autism – has proven its effectiveness in children with autism. In 2008, the Journal of Child Neurology published a follow-up study that found that children with autism who had been exposed to similar behaviors


[{'generated_text': 'machine learning\n\nUsing the same system that has been used ever since with Microsoft Excel, Microsoft has demonstrated a powerful way to use and create a variety of Excel documents using the Microsoft OneDrive and Microsoft OneMover. The company has started supporting the'}]

In [6]:
docs = [generated_text1[0]['generated_text'], generated_text2[0]['generated_text'], generated_text3[0]['generated_text']]

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
# Download NLTK stopwords
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))


cleaned_corpus = []
for document in docs:
    document = document.lower()

    document = re.sub(r'[^\w\s]|[\d]', '', document)

    words = nltk.word_tokenize(document)

    filtered_words = [word for word in words if word.lower() not in stop_words]

    cleaned_sentence = ' '.join(filtered_words)
    cleaned_corpus.append(cleaned_sentence)

cleaned_corpus

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['machine learning using system used ever since microsoft excel microsoft demonstrated powerful way use create variety excel documents using microsoft onedrive microsoft onemover company started supporting',
 'machine learning social networks said id also like mention interesting ai system far existence discovered social networks already interacting even less processing time',
 'machine learning cognitivebehavioral interventions children autism proven effectiveness children autism journal child neurology published followup study found children autism exposed similar behaviors']

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tr_idf_model  = TfidfVectorizer()
tf_idf_vector = tr_idf_model.fit_transform(cleaned_corpus)

In [10]:
print(type(tf_idf_vector), tf_idf_vector.shape)
# <class'scipy.sparse.csr.csr_matrix'>(3,14)

<class 'scipy.sparse._csr.csr_matrix'> (3, 55)


In [18]:
import numpy as np
import pandas as pd
tfidf_array = tf_idf_vector.toarray()

normalized_tfidf_array = np.zeros_like(tfidf_array)

for i in range(len(cleaned_corpus)):
    sum_of_squares = np.sqrt(np.sum(tfidf_array[i] ** 2))
    normalized_tfidf_array[i] = tfidf_array[i] / sum_of_squares

# Get feature names (words) from TfidfVectorizer
words_set = tr_idf_model.get_feature_names_out()

df_normalized_tfidf = pd.DataFrame(normalized_tfidf_array, columns=words_set)
print(df_normalized_tfidf)

         ai   already      also    autism  behaviors     child  children  \
0  0.000000  0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
1  0.198905  0.198905  0.198905  0.000000   0.000000  0.000000  0.000000   
2  0.000000  0.000000  0.000000  0.524642   0.174881  0.174881  0.524642   

   cognitivebehavioral   company    create  ...   started     study  \
0             0.000000  0.157571  0.157571  ...  0.157571  0.000000   
1             0.000000  0.000000  0.000000  ...  0.000000  0.000000   
2             0.174881  0.000000  0.000000  ...  0.000000  0.174881   

   supporting    system      time       use      used     using   variety  \
0    0.157571  0.119837  0.000000  0.157571  0.157571  0.315142  0.157571   
1    0.000000  0.151272  0.198905  0.000000  0.000000  0.000000  0.000000   
2    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

        way  
0  0.157571  
1  0.000000  
2  0.000000  

[3 rows x 55 columns]


In [19]:
words_set = tr_idf_model.get_feature_names_out()
print(words_set)

['ai' 'already' 'also' 'autism' 'behaviors' 'child' 'children'
 'cognitivebehavioral' 'company' 'create' 'demonstrated' 'discovered'
 'documents' 'effectiveness' 'even' 'ever' 'excel' 'existence' 'exposed'
 'far' 'followup' 'found' 'id' 'interacting' 'interesting' 'interventions'
 'journal' 'learning' 'less' 'like' 'machine' 'mention' 'microsoft'
 'networks' 'neurology' 'onedrive' 'onemover' 'powerful' 'processing'
 'proven' 'published' 'said' 'similar' 'since' 'social' 'started' 'study'
 'supporting' 'system' 'time' 'use' 'used' 'using' 'variety' 'way']


In [20]:
import pandas as pd

df_tf_idf = pd.DataFrame(tfidf_array, columns = words_set)

df_tf_idf

,ai,already,also,autism,behaviors,child,children,cognitivebehavioral,company,create,...,started,study,supporting,system,time,use,used,using,variety,way
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.157571,0.157571,...,0.157571,0.000000,0.157571,0.119837,0.000000,0.157571,0.157571,0.315142,0.157571,0.157571
1,0.198905,0.198905,0.198905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.151272,0.198905,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.524642,0.174881,0.174881,0.524642,0.174881,0.000000,0.000000,...,0.000000,0.174881,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


##Computing TF-IDF from scratch


In [21]:
import numpy as np

n_docs = len(cleaned_corpus)
n_words_set = len(words_set)

df_tf = pd.DataFrame(np.zeros((n_docs, n_words_set)), columns=words_set)

# Compute Term Frequency (TF)
for i in range(n_docs):
    words = cleaned_corpus[i].replace('\n','').split(' ')
    for w in words:
        df_tf[w][i] = df_tf[w][i] + (1 / len(words))

df_tf

,ai,already,also,autism,behaviors,child,children,cognitivebehavioral,company,create,...,started,study,supporting,system,time,use,used,using,variety,way
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.038462,...,0.038462,0.000000,0.038462,0.038462,0.000000,0.038462,0.038462,0.076923,0.038462,0.038462
1,0.043478,0.043478,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.043478,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.136364,0.045455,0.045455,0.136364,0.045455,0.000000,0.000000,...,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
print("IDF of: ")

idf = {}

for w in words_set:
    k = 0    # number of documents in the corpus that contain this word

    for i in range(n_docs):
        if w in cleaned_corpus[i].split():
            k += 1

    idf[w] =  np.log10(n_docs + 1 / k +1) + 1

    print(f'{w:>15}: {idf[w]:>10}' )

IDF of: 
             ai: 1.6989700043360187
        already: 1.6989700043360187
           also: 1.6989700043360187
         autism: 1.6989700043360187
      behaviors: 1.6989700043360187
          child: 1.6989700043360187
       children: 1.6989700043360187
cognitivebehavioral: 1.6989700043360187
        company: 1.6989700043360187
         create: 1.6989700043360187
   demonstrated: 1.6989700043360187
     discovered: 1.6989700043360187
      documents: 1.6989700043360187
  effectiveness: 1.6989700043360187
           even: 1.6989700043360187
           ever: 1.6989700043360187
          excel: 1.6989700043360187
      existence: 1.6989700043360187
        exposed: 1.6989700043360187
            far: 1.6989700043360187
       followup: 1.6989700043360187
          found: 1.6989700043360187
             id: 1.6989700043360187
    interacting: 1.6989700043360187
    interesting: 1.6989700043360187
  interventions: 1.6989700043360187
        journal: 1.6989700043360187
       learning

In [23]:
df_tf_idf = df_tf.copy()

for w in words_set:
    for i in range(n_docs):
        df_tf_idf[w][i] = df_tf[w][i] * idf[w]

df_tf_idf

,ai,already,also,autism,behaviors,child,children,cognitivebehavioral,company,create,...,started,study,supporting,system,time,use,used,using,variety,way
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.065345,0.065345,...,0.065345,0.000000,0.065345,0.063585,0.000000,0.065345,0.065345,0.13069,0.065345,0.065345
1,0.073868,0.073868,0.073868,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.071879,0.073868,0.000000,0.000000,0.00000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.231678,0.077226,0.077226,0.231678,0.077226,0.000000,0.000000,...,0.000000,0.077226,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
